In [24]:
import numpy as np
import pandas as pd
import os.path, matplotlib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import time
from sklearn.decomposition import PCA
import matplotlib.pyplot
from mpl_toolkits.mplot3d import Axes3D
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

In [8]:
data = pd.read_csv('data/data.csv')

In [36]:
# no encoding
label = data.label
df = data.drop(['label','filename'], axis=1)
scaler = StandardScaler()
scaler.fit(df)
df_scaled = scaler.transform(df)
# df_scaled = df
x_train, x_test, y_train, y_test = train_test_split(df_scaled, label, test_size=0.2)

In [37]:
# linear kernel svc
linsvc = SVC(kernel='linear')
start_time = time.time()
linsvc_model = OneVsRestClassifier(linsvc).fit(x_train, y_train)
print("elapsed: {}".format(time.time() - start_time))
# linsvc_pred = linsvc_model.predict(x_test)
print(linsvc_model.score(x_test, y_test))
# linsvc_pred = linsvc_model.predict(x_train)
print(linsvc_model.score(x_train, y_train))

elapsed: 0.1360468864440918
0.56
0.68875


In [66]:
rfc_clf = RandomForestClassifier(max_depth=50, n_estimators= 100, max_features="sqrt")
start_time = time.time()
rfc_model = rfc_clf.fit(x_train, y_train)
print("elapsed: {}".format(time.time() - start_time))
print(rfc_model.score(x_test, y_test))
print(rfc_model.score(x_train, y_train))

elapsed: 0.2786979675292969
0.63
0.99875


In [157]:
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 1000, num = 10)]
max_depth = [int(x) for x in np.linspace(start = 10, stop = 1000, num = 10)]

# n_estimators = [int(x) for x in np.linspace(start = 100, stop = 1000, num = 10)]
# max_depth = [int(x) for x in np.linspace(start = 100, stop = 500, num = 5)]
max_features = ['sqrt','log2', None]
rfc_params = {
    'n_estimators': n_estimators,
    'max_depth': max_depth,
    'max_features': max_features,
}
rfc_params

{'n_estimators': [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000],
 'max_depth': [10, 120, 230, 340, 450, 560, 670, 780, 890, 1000],
 'max_features': ['sqrt', 'log2', None]}

In [158]:
rfc = RandomForestClassifier()
rfc_grid = GridSearchCV(rfc, rfc_params, n_jobs=-1, cv=50)
start_time = time.time()
rfc_grid.fit(x_train, y_train)
print("elapsed: {}".format(time.time() - start_time))


KeyboardInterrupt: 

In [67]:
print(rfc_grid.best_score_)
print(rfc_grid.best_params_)

0.6425000000000001
{'max_depth': 100, 'max_features': 'sqrt', 'n_estimators': 200}


In [1]:
iris = datasets.load_iris()
parameters = {'kernel':('linear', 'rbf'), 'C':[1, 10]}
svc = svm.SVC()
clf = GridSearchCV(svc, parameters)
clf.fit(iris.data, iris.target)
sorted(clf.cv_results_.keys())

['mean_fit_time',
 'mean_score_time',
 'mean_test_score',
 'param_C',
 'param_kernel',
 'params',
 'rank_test_score',
 'split0_test_score',
 'split1_test_score',
 'split2_test_score',
 'split3_test_score',
 'split4_test_score',
 'std_fit_time',
 'std_score_time',
 'std_test_score']

In [5]:
clf.cv_results_

{'mean_fit_time': array([0.00066051, 0.00051756, 0.00035329, 0.00040765]),
 'std_fit_time': array([6.09025289e-04, 9.95576608e-05, 2.05783825e-05, 7.15192792e-05]),
 'mean_score_time': array([0.00023985, 0.00027957, 0.0002028 , 0.00021877]),
 'std_score_time': array([5.60124354e-05, 4.74687978e-05, 2.17784638e-06, 1.71207782e-05]),
 'param_C': masked_array(data=[1, 1, 10, 10],
              mask=[False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_kernel': masked_array(data=['linear', 'rbf', 'linear', 'rbf'],
              mask=[False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'C': 1, 'kernel': 'linear'},
  {'C': 1, 'kernel': 'rbf'},
  {'C': 10, 'kernel': 'linear'},
  {'C': 10, 'kernel': 'rbf'}],
 'split0_test_score': array([0.96666667, 0.96666667, 1.        , 0.96666667]),
 'split1_test_score': array([1.        , 0.96666667, 1.        , 1.        ]),
 'split2_test_score': array([0.96666667, 0.96666667, 0

In [98]:
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 500, num = 1)]
max_depth = [int(x) for x in np.linspace(start = 100, stop = 500, num = 1)]
max_features = ['sqrt','log2', None]
params = {
    'n_estimators': n_estimators,
    'max_depth': max_depth,
    'max_features': max_features,
}
params

{'n_estimators': [100],
 'max_depth': [100],
 'max_features': ['sqrt', 'log2', None]}

In [73]:
import pickle
filename = 'GridSearchRFC.sav'
pickle.dump(rfc_grid, open(filename, 'wb'))

In [74]:
temp = pickle.load(open(filename, 'rb'))

In [75]:
temp.best_params_

{'max_depth': 100, 'max_features': 'sqrt', 'n_estimators': 200}

In [90]:
temp.best_index_

7

In [91]:
rfc = RandomForestClassifier(**temp.best_params_)
rfc.fit(x_train, y_train)
rfc.score(x_test, y_test)

0.63

In [167]:
# C, kernel, gamma, degree
C = np.logspace(-2, 4, 7)
gamma = np.logspace(-3, 3, 7)
kernel = ['poly', 'rbf', 'sigmoid']
params = {
    'C': C,
    'gamma': gamma,
    'kernel': kernel,
}

In [165]:
params = {
    'C': C,
    'gamma': gamma,
    'kernel': kernel,
}

In [166]:
params

{'C': array([1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03, 1.e+04]),
 'gamma': array([1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03]),
 'kernel': ['poly', 'rbf', 'sigmoid']}

In [160]:
C = np.logspace(-2, 4, 7)

In [162]:
gamma = np.logspace(-3, 3, 7)

In [164]:
kernel = ['poly', 'rbf', 'sigmoid']